<p style="font-size:2.2em; color:white; padding:15px; border-bottom: 5px solid #ffb701; background:#143157; text-align:center;"> 
    Sales Profitability  <br> 
    <span style="font-size:.55em; color: #11a1f5;"> Sales Anaysis </span>
</p>

<p style="font-family:Calibri; font-size:1.75em; color:white; padding:5px 5px; text-align: left; height:50px; border-bottom: 3px solid orange; margin:-7px 0px;"></p>

<p style="font-size:2.2em; padding:10px; border-bottom: 5px solid #ffb701; background:#492863;
          text-align:center"> 
    <span style="color: #debef7;"> Update Sales Revenue Data  </span> <br>
</p>


<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 1 - </span>
   <span style="font-size:1.4em; color:#e39607;">Get Company and Year - </span> 
   <span style="font-size:1.2em; color:#209ebd;">to Load Data </span>
</p> 

In [213]:
###---Standard Code to load Data-------------------------------
import pandas as pd
import numpy as np
dfStd = pd.read_csv('data/basInfo.csv')
Year = str(dfStd.loc[0, 'year'])
coFolder = str(dfStd.loc[0, 'coFolder'])

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 2 - </span>
   <span style="font-size:1.4em; color:#e39607;">Read siBasic.csv siAddi.csv File</span> 
</p> 

In [214]:
basicPath = "Data/" +coFolder+ "/" +Year+ "/F1_Sales/Invoices/siBasic.csv"
siBasic = pd.read_csv(basicPath, index_col=False, encoding='unicode_escape').fillna('')

addiPath = "Data/" +coFolder+ "/" +Year+ "/F1_Sales/Invoices/siAddi.csv"
siAddi = pd.read_csv(addiPath, index_col=False, encoding='unicode_escape').fillna('')

In [215]:
### siBasic Remove Duplicate Invoice and Deleted Record...convert date into standard date formate........
siBasic = siBasic.drop_duplicates(subset=['siRef'], keep='last')   ### Remove previous record
siBasic = siBasic.drop(siBasic.loc[siBasic['action']=='Deleted'].index)  ### Remove Deleted row
siBasic['siDat'] = pd.to_datetime(siBasic['siDat']).dt.strftime('%Y-%m-%d')  ### convert date to standard Date
siBasic['unique'] = siBasic['siRef'].astype(str)+" "+siBasic['traDate']  ### Creat a unique column

In [216]:
siAddi["unique"] = siAddi['siRef'].astype(str)+" "+siAddi['traDate']
###---- Merge to filter out those data which Transaction Date is not matching with qotBasic
mergedSi = pd.merge(siAddi, siBasic['unique'], on='unique', how='inner')

In [217]:
siAddiCom = pd.merge(mergedSi, siBasic[['siRef', 'siDat',  'cusCode', 'cusName', 'spCode', 'spCName', 'dlRef']], on='siRef', how='left')

In [218]:
siAddiCom['dlRef'] = siAddiCom['dlRef'].astype(int).astype(str) ### convert 2300003.0 to 2300003

In [219]:
siAddiCom['unique'] = siAddiCom['dlRef'].astype(str)+' '+siAddi['itmCod'].astype(str)

In [220]:
siAddiCom['Sales'] = siAddiCom['tot']- siAddi['vat']

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 3 - </span>
   <span style="font-size:1.4em; color:#e39607;">Read dlBasic.csv dlAddi.csv File</span> 
</p> 

In [221]:
req_cols = ['dlRef','traDate','action']
basicPath = "Data/" +coFolder+ "/" +Year+ "/F1_Sales/Deliveries/dlBasic.csv"
dlBasic = pd.read_csv(basicPath, usecols=req_cols, index_col=False, encoding='unicode_escape').fillna('')

req_cols = ['dlRef','itmCod','wac','traDate','action']
addiPath = "Data/" +coFolder+ "/" +Year+ "/F1_Sales/Deliveries/dlAddi.csv"
dlAddi = pd.read_csv(addiPath, usecols=req_cols, index_col=False, encoding='unicode_escape').fillna('')

In [222]:
### siBasic Remove Duplicate Invoice and Deleted Record...convert date into standard date formate........
dlBasic = dlBasic.drop_duplicates(subset=['dlRef'], keep='last')   ### Remove previous record
dlBasic = dlBasic.drop(dlBasic.loc[dlBasic['action']=='Deleted'].index)  ### Remove Deleted row
dlBasic['uniqueA'] = dlBasic['dlRef'].astype(str)+" "+dlBasic['traDate']  ### Creat a unique column

In [223]:
dlAddi["uniqueA"] = dlAddi['dlRef'].astype(str)+" "+dlAddi['traDate']
###---- Merge to filter out those data which Transaction Date is not matching with qotBasic
mergedDl = pd.merge(dlAddi, dlBasic['uniqueA'], on='uniqueA', how='inner')

In [224]:
mergedDl['unique'] = mergedDl['dlRef'].astype(str)+' '+mergedDl['itmCod'].astype(str)

In [225]:
### Get 'wac' column and merge with  salAddi-----------
MergedDf = pd.merge(siAddiCom, mergedDl[['unique', 'wac']], on='unique', how='left')

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 4 - </span>
   <span style="font-size:1.4em; color:#e39607;">Read ItemSalPrice.csv File</span> 
</p> 

In [226]:
req_cols = ['itmCode','brand','supplier']
salPricePath = "Data/" +coFolder+ "/" +Year+ "/H1_Items/Items/ItemSalPrice.csv"
salPrice = pd.read_csv(salPricePath, usecols=req_cols, index_col=False, encoding='unicode_escape').fillna('')

In [227]:
salPrice = salPrice.rename(columns={'itmCode': 'itmCod'})

In [228]:
salPrice = salPrice.drop_duplicates(subset=['itmCod'], keep='last')   ### Remove previous record

In [229]:
### Get 'wac' column and merge with  salAddi-----------
Merged = pd.merge(MergedDf,salPrice[['itmCod', 'supplier']], on='itmCod', how='left')

In [230]:
### Save the data in qotStat.csv File...
statPath =  "Data/" +coFolder+ "/" +Year+ "/F1_Sales/Analysis/Sales Report.csv"
Merged.to_csv(statPath, index=False)

<p style="font-family:Calibri; font-size:1.75em; color:white; padding:5px 5px; text-align: left; height:50px; border-bottom: 3px solid orange; margin:-7px 0px;"></p>

<p style="font-size:2.2em; padding:10px; border-bottom: 5px solid #ffb701; background:#492863;
          text-align:center"> 
    <span style="color: #debef7;"> Revenue Graph  </span> <br>
</p>


<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 1 - </span>
   <span style="font-size:1.4em; color:#e39607;">Get Company and Year - </span> 
   <span style="font-size:1.2em; color:#209ebd;">to Load Data </span>
</p> 

In [1]:
###---Standard Code to load Data-------------------------------
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_columns', 85) # set Jupeter note book to dispay 85 columns
dfStd = pd.read_csv('data/basInfo.csv')
Year = str(dfStd.loc[0, 'year'])
coFolder = str(dfStd.loc[0, 'coFolder'])

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 2 - </span>
   <span style="font-size:1.4em; color:#e39607;">Read siBasic.csv siAddi.csv File</span> 
</p> 

In [2]:
basicPath = "Data/" +coFolder+ "/" +Year+ "/F1_Sales/Analysis/Sales Report.csv"
salReport = pd.read_csv(basicPath, index_col=False, encoding='unicode_escape').fillna('')

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 3 - </span>
   <span style="font-size:1.4em; color:#e39607;">Monthly Sales and Gross Profit</span> 
</p> 

In [4]:
salReport['GP'] = salReport['Sales'] - salReport['wac']

In [6]:
salReport

,id,siRef,sno,itmCod,desc,qty,price,disc,tot,vat,traDate,action,unique,siDat,cusCode,cusName,spCode,spCName,dlRef,Sales,wac,supplier,GP
0,2300001,2300001,1,10001000,Microneedling RF,1,31465,2465,"30,450","1,450",2023-5-29 22-27-46,,2300003 10001000,2023-01-24,"22,310,093",Women's Medical Center (Dr. Ameena),370,Ariel Abapo Naciogoyo,2300003,"29,000","22,665",Adss Laser Co,"6,335"
1,2300001,2300001,2,10001001,Microneedling RF - HandPiece,1,4725,650,"4,279",204,2023-5-29 22-27-46,,2300003 10001001,2023-01-24,"22,310,093",Women's Medical Center (Dr. Ameena),370,Ariel Abapo Naciogoyo,2300003,"4,075","3,185",Adss Laser Co,890
2,2300002,2300002,1,10004000,RF Fractional CO2 Laser,1,81900,2900,"82,950","3,950",2023-5-29 22-28-1,,2300004 10004000,2023-01-29,"22,310,055",Al Noor Pharmacy,366,Franco Viola Parota,2300004,"79,000","58,519",Skin for Life LLC,"20,481"
3,2300002,2300002,2,10004001,RF Fractional CO2 Laser - Handpiece,1,7700,300,"7,770",370,2023-5-29 22-28-1,,2300004 10004001,2023-01-29,"22,310,055",Al Noor Pharmacy,366,Franco Viola Parota,2300004,"7,400","5,481",Skin for Life LLC,"1,919"
4,2300003,2300003,1,10018001,SVTA Radio Hand Pice 1,2,1680,360,"3,150",150,2023-5-29 22-28-14,,2300001 10018001,2023-02-01,"22,310,182",Spaces Spa Center,370,Ariel Abapo Naciogoyo,2300001,"3,000","2,438",SPA and Co LLC,562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,2300265,2300265,2,10018005,SVTA Radio Hand Pice 5,1,2240,140,"2,205",105,2023-6-3 13-47-7,,2300266 10018005,2023-12-31,"22,311,809",Carmen Beauty Salon,994,Michael Cadiang Manza,2300266,"2,100","1,546",SPA and Co LLC,554
578,2300266,2300266,1,10001000,Microneedling RF,1,31465,2465,"30,450","1,450",2023-6-12 15-27-33,,2300061 10001000,2023-05-01,"22,310,102",Health & Aesthetic Medical Center,370,Ariel Abapo Naciogoyo,2300061,"29,000","22,421",Adss Laser Co,"6,579"
579,2300266,2300266,2,10001001,Microneedling RF - HandPiece,2,4725,650,"9,240",440,2023-6-12 15-27-33,,2300061 10001001,2023-05-01,"22,310,102",Health & Aesthetic Medical Center,370,Ariel Abapo Naciogoyo,2300061,"8,800","6,804",Adss Laser Co,"1,996"
580,2300267,2300267,1,10016000,My Ultrasonic,2,9800,1800,"18,690",890,2023-6-12 15-27-52,,2300270 10016000,2023-06-15,"22,311,988",Glamour Care Clinics LLC,"1,138",Jessie Tolntion Matitu,2300270,"17,800","13,940",Ultrasonic and Co LLC,"3,860"


In [56]:
salReport['siDat'] = pd.to_datetime(salReport['siDat'])
salReport['month'] = salReport['siDat'].dt.month
salReport['year'] = salReport['siDat'].dt.year

In [51]:
monSales = salReport.groupby(['year', 'month'])[['Sales', 'wac', 'GP']].sum().reset_index()
monSales['Sales'] = monSales['Sales'] / 1000
monSales['wac'] = monSales['wac'] / 1000
monSales['GP'] = monSales['GP'] / 1000

In [43]:
totSal = monSales['Sales'].sum()

In [44]:
monSales['percentage'] = (monSales['Sales'] / totSal) * 100
monSales['percentage'] = monSales['percentage'].round(1)

monSales['gpPer'] = (monSales['GP'] / monSales['Sales']) * 100
monSales['gpPer'] = monSales['gpPer'].round(1)

In [35]:
### [1.85, 6.78, 10.25, 8.65, 9.15, 7.21, 8.37, 9.04, 9.37, 8.13, 10.31, 10.88]
perList = monSales['percentage'].tolist()

In [36]:
monSales = monSales.to_dict(orient="records")

<p style="padding:5px; border-bottom:3px solid #6d79a1; background:#e9edf7; margin:5px 0 -7px 20px;"> 
   <span style="font-size:1.4em; color:#202f61;" >Step 4 - </span>
   <span style="font-size:1.4em; color:#e39607;">Top 10 Customers</span> 
</p> 

In [7]:
topCust = salReport.groupby(['cusName'])[['Sales','GP']].sum().reset_index()

In [11]:
top10Cust = topCust.nlargest(10, 'Sales')
top10Cust = top10Cust.sort_values(by='Sales', ascending=False)

top10Cust['percentage'] = (top10Cust['GP'] / top10Cust['Sales']) * 100
top10Cust['percentage'] = top10Cust['percentage'].round(1)

top10Cust

,cusName,Sales,GP,percentage
4,Al Jameya Pharmacy,"869,380","199,355",23
37,Sahar Pharmacy,"745,605","160,305",22
23,Gulf Beauty Salon - Abu Dhabi,"718,895","150,295",21
20,German Clinic FZ LLC,"617,030","100,330",16
39,Spaces Spa Center,"616,960","104,585",17
16,Delta Pharmacy,"572,400","122,400",21
43,Women's Medical Center (Dr. Ameena),"557,755","140,855",25
5,Al Noor Pharmacy,"550,360","126,260",23
19,Flora Beauty Spa,"516,800","109,300",21
3,Al Dar Medical Center,"440,600","73,600",17
